In [ ]:
!pip install lime

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# 필요한 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 토크나이저 로드
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# 모델 로드 및 가중치 적용
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
model.load_state_dict(torch.load("/content/drive/MyDrive/프로젝트NLP/학습모델weight/best_model.pth"))
model.to(device)

class_weights = torch.load("/content/drive/MyDrive/프로젝트NLP/학습모델weight/class_weights.pth").to(device)

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from lime.lime_text import LimeTextExplainer
import numpy as np

# LIME 설명자를 생성합니다.
explainer = LimeTextExplainer(class_names=["no", "yes"])

#gpu터져서 cpu사용
device = torch.device("cpu")
model.to(device)

# 모델의 예측 함수를 정의합니다.
def predictor(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=160, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()
    return probs

lime_summary = []
for i, text in enumerate(test['text'][1000:]):
    exp = explainer.explain_instance(
        text,
        predictor,
        num_features=10,
        num_samples=100
    )
    feature_contributions = ", ".join([f"{feature}:{round(contribution, 3)}" for feature, contribution in exp.as_list()])

    lime_summary.append({
        "text":text,
        "feature_contributions": feature_contributions
    })

lime_summary_df = pd.DataFrame(lime_summary)
# lime_summary_df.to_excel('/content/drive/MyDrive/프로젝트NLP/lime_summary_test2.xlsx', index=False)